In [1]:
import tensorflow as tf
import pandas as pd 
from tensorflow import keras

NUM_CLASSES = 10
BATCH_SIZE = 32

In [2]:
train_df = pd.read_csv("data/train.csv").sample(frac=1, random_state=42)
val_df = pd.read_csv("data/val.csv")
test_df = pd.read_csv("data/test.csv")

train_df.head(32)

,image_path,label,subject,camera
8602,data/Camera 2/train/c2/122510.jpg,2,44,2
9558,data/Camera 2/train/c8/127615.jpg,8,44,2
6827,data/Camera 1/train/c8/4100000000.jpg,8,7,1
7614,data/Camera 1/train/c9/208700000000.jpg,9,20,1
1446,data/Camera 1/train/c0/3519.jpg,0,25,1
3131,data/Camera 1/train/c2/250.jpg,2,3,1
4906,data/Camera 1/train/c4/978000.jpg,4,18,1
3281,data/Camera 1/train/c2/445.jpg,2,7,1
1149,data/Camera 1/train/c0/3045.jpg,0,21,1
2471,data/Camera 1/train/c1/205.jpg,1,27,1


In [3]:
def preprocess(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=0)
    img = tf.image.resize(img, (224, 224))
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)

    return img, label

In [4]:
image_path_train = train_df['image_path']
image_path_test = test_df['image_path']
image_path_val = val_df['image_path']
label_train = train_df['label']
label_test = test_df['label']
label_val = val_df['label']

In [5]:
#train dataset
train_data = tf.data.Dataset.from_tensor_slices((image_path_train, label_train))
train_data = train_data.map(preprocess, num_parallel_calls=6).shuffle(int(1e3)).batch(BATCH_SIZE).prefetch(1)
#validation dataset
val_data = tf.data.Dataset.from_tensor_slices((image_path_val, label_val))
val_data = val_data.map(preprocess, num_parallel_calls=6).batch(BATCH_SIZE).prefetch(1)
#test dataset
test_data = tf.data.Dataset.from_tensor_slices((image_path_test, label_test))
test_data = test_data.map(preprocess, num_parallel_calls=6).batch(BATCH_SIZE).prefetch(1)

2021-12-17 15:14:50.195456: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 15:14:50.210456: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 15:14:50.210960: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-17 15:14:50.211842: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
from datetime import datetime
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)
with file_writer.as_default():
    # Don't forget to reshape.
    for sample, _ in train_data.take(1):
        pass
    
    tf.summary.image("Batch training data examples", sample, max_outputs=32, step=0)

In [7]:
base_model = keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(NUM_CLASSES)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
Total params: 2,270,794
Trainable params: 2,236,682
Non-trainable params: 34,112
_________________________________________________________________


In [8]:
opt = keras.optimizers.Nadam(learning_rate=1e-5)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor='val_accuracy')
]

In [ ]:
history = model.fit(train_data, epochs=100, validation_data=val_data, callbacks=callbacks)

Epoch 1/100


2021-12-17 15:15:27.424416: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2021-12-17 15:15:27.750451: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


183/308 [================>.............] - ETA: 50s - loss: 8.2604 - accuracy: 0.1166

In [ ]:
sample.shape

In [ ]:
base_model.trainable = True
opt = keras.optimizers.Nadam(learning_rate=5e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_data, epochs=10, validation_data=val_data)

In [ ]:
history.history

In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_data, batch_size=64)
print("test loss, test acc:", results)

In [ ]:
best_val_loss 
num_epochs = 10
epoch = 0

while epoch < num_epochs:
    model.train_on_batch(train_data)  
    val_loss = model.evaluate(val_data)

    if val_loss < best_val_loss:
        model.save(best_model_path) # OR model.save_weights()
        print("Best model w/ val loss {} saved to {}".format(val_loss, best_model_path))
    
    epoch += 1